#Telecom Domain ReadOps Assignment
This notebook contains assignments to practice Spark read options and Databricks volumes. <br>
Sections: Sample data creation, Catalog & Volume creation, Copying data into Volumes, Path glob/recursive reads, toDF() column renaming variants, inferSchema/header/separator experiments, and exercises.<br>

![](https://fplogoimages.withfloats.com/actual/68009c3a43430aff8a30419d.png)
![](https://theciotimes.com/wp-content/uploads/2021/03/TELECOM1.jpg)

##First Import all required libraries & Create spark session object

##1. Write SQL statements to create:
1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol
4. Using dbutils.fs.mkdirs, create folders:<br>
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):<br>
a. Volume vs DBFS/FileStore<br>
b. Why production teams prefer Volumes for regulated data<br>

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS telecom_catalog_assign;
CREATE SCHEMA IF NOT EXISTS telecom_catalog_assign.landing_zone;
CREATE VOLUME telecom_catalog_assign.landing_zone.landing_vol;


In [0]:
%sql
DROP CATALOG IF EXISTS telecom_catalog_assign CASCADE;

In [0]:
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/region2")

In [0]:
# We can create multiple folders in one go by using a loop, instead of writing dbutils.fs.mkdirs() one by one.

folders = ['/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer','/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage','/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1 ','/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2']

for folder in folders:
  dbutils.fs.mkdirs(folder)

##Data files to use in this usecase:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''

In [0]:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''
usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''
tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
5002|104|TWR06|-75|2025-01-10 11:01:12
5003|104|TWR02|-80|2025-01-10 11:01:12
5005|104|TWR03|-75|2025-01-10 11:01:12
5006|104|TWR04|-75|2025-01-10 11:01:12
'''



##2. Filesystem operations
1. Write code to copy the above datasets into your created Volume folders:
Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/

2. Write a command to validate whether files were successfully copied

In [0]:
# 2.1. Write code to copy the above datasets into your created Volume folders:
'''Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/'''

dbutils.fs.put('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv', customer_csv, True)
dbutils.fs.put('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv', usage_tsv, True)
dbutils.fs.put('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_logs_region1.csv',tower_logs_region1,True)
dbutils.fs.put('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/tower_logs_region1.csv',tower_logs_region1,True)

In [0]:
# 2.2. Write a command to validate whether files were successfully copied

print(dbutils.fs.ls('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/'))
print(dbutils.fs.ls('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/'))
print(dbutils.fs.ls('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1'))
print(dbutils.fs.ls('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2'))

##3. Directory Read Use Cases
1. Read all tower logs using:
Path glob filter (example: *.csv)
Multiple paths input
Recursive lookup

2. Demonstrate these 3 reads separately:
Using pathGlobFilter
Using list of paths in spark.read.csv([path1, path2])
Using .option("recursiveFileLookup","true")

3. Compare the outputs and understand when each should be used.

In [0]:
# 3.1. Read all tower logs using Path glob filter (example: *.csv),Multiple paths input,Recursive lookup:
tower_df1 = spark.read.csv([
  "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1",
  "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2"], 
  header=True, 
  inferSchema=True, 
  sep="|", 
  pathGlobFilter="*.csv", 
  recursiveFileLookup=True
  )
display(tower_df1)

# 3.2. Demonstrate these 3 reads separately:

# 3.2.1. Using pathGlobFilter only
tower_df2 = spark.read.csv(
  "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region*",  # We can use a wild card (*) to load data from all the files or folders that match a path pattern.
  header=True, 
  inferSchema=True, 
  sep="|", 
  pathGlobFilter="*.csv"  # We are using pathGlobFilter is to filter out only the csv files from the folder and also we can load multiple files at the same time.
  )
display(tower_df2)

# 3.2.2. Using list of paths in spark.read.csv([path1, path2])
tower_df3 = spark.read.csv([
  "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/"],
  header=True, 
  inferSchema=True, 
  sep="|", 
  pathGlobFilter="*.csv"
  )                     # We can mention the different paths in the list of csv and we can load the data from different folders
display(tower_df3)

# 3.2.3. .option("recursiveFileLookup", "true") with multiple paths
tower_df4 = spark.read.option(
    "header","True").option(
      "inferSchema","True").option(
        "sep","|").option(
          "pathGlobFilter","*.csv").option(
            "recursiveFileLookup","true").csv(
              "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")
       # if we mention main folder name in the path and recursiveFileLookup=True, it will load all the data from the subfolders
display(tower_df4)   

##4. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

##5. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 

## Spark Write Operations using 
- csv, json, orc, parquet, delta, saveAsTable, insertInto, xml with different write mode, header and sep options

##6. Write Operations (Data Conversion/Schema migration) – CSV Format Usecases
1. Write customer data into CSV format using overwrite mode
2. Write usage data into CSV format using append mode
3. Write tower data into CSV format with header enabled and custom separator (|)
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#6.1. Write customer data into CSV format using overwrite mode
customer_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv",header=False,inferSchema=True,sep=",")
customer_df.write.csv(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_out_csv",sep=",",header=True,mode="overwrite")
display(customer_df)

In [0]:
#6.2. Write usage data into CSV format using append mode
usage_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv", header=True, inferSchema=True, sep="\t")
usage_df.write.csv(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_out_csv", sep=",", header=True, mode="append")
display(usage_df)

In [0]:
#6.3. Write tower data into CSV format with header enabled and custom separator (|)
tower_df1.write.csv(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_out_csv", sep="|", header=True, mode="overwrite")
display(tower_df1)

In [0]:
#6.4. Read the tower data in a dataframe and show only 5 rows.
tower_df2 = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_out_csv", header=True, inferSchema=True, sep="|")
display(tower_df2.limit(5))

#6.5. Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++
display(
    dbutils.fs.ls(
        "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_out_csv"
    )
)

##7. Write Operations (Data Conversion/Schema migration)– JSON Format Usecases
1. Write customer data into JSON format using overwrite mode
2. Write usage data into JSON format using append mode and snappy compression format
3. Write tower data into JSON format using ignore mode and observe the behavior of this mode
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#7.1. Write customer data into JSON format using overwrite mode
customer_df.write.json(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_out_json", mode="overwrite")
display(customer_df)

#7.2. Write usage data into JSON format using append mode and snappy compression format
usage_df.write.json(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_out_json", mode="append", compression="snappy")
display(usage_df)

#7.3. Write tower data into JSON format using ignore mode and observe the behavior of this mode
tower_df2.write.json(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_out_json", mode="ignore")
display(tower_df2)

#7.4. Read the tower data in a dataframe and show only 5 rows.
tower_df3 = spark.read.json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_out_json")
display(tower_df3.limit(5))


##8. Write Operations (Data Conversion/Schema migration) – Parquet Format Usecases
1. Write customer data into Parquet format using overwrite mode and in a gzip format
2. Write usage data into Parquet format using error mode
3. Write tower data into Parquet format with gzip compression option
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
# 8.1. Write customer data into Parquet format using overwrite mode and in a gzip format
customer_df.write.parquet(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_out_json", mode="overwrite", compression="gzip")

#8.2. Write usage data into Parquet format using error mode
usage_df.write.parquet(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_out_parquet", mode="error")

#8.3. Write tower data into Parquet format with gzip compression option
tower_df2.write.parquet(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_out_parquet", mode="overwrite",compression="gzip")

#8.4. Read the usage data in a dataframe and show only 5 rows.
tower_df3 = spark.read.json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_out_json")
display(tower_df3.limit(5))

##9. Write Operations (Data Conversion/Schema migration) – Orc Format Usecases
1. Write customer data into ORC format using overwrite mode
2. Write usage data into ORC format using append mode
3. Write tower data into ORC format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
# 9.1. Write customer data into ORC format using overwrite mode
customer_df.write.orc(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_out_orc", mode="overwrite")

#9.2. Write usage data into ORC format using append mode
usage_df.write.orc(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_out_orc", mode="append")

#9.3. Write tower data into ORC format and see the output file structure
tower_df2.write.orc(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_out_orc", mode="overwrite")
# List files in the output directory
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_out_orc")

#9.4. Read the usage data in a dataframe and show only 5 rows.
tower_df3 = spark.read.json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_out_json")
display(tower_df3.limit(5))

##10. Write Operations (Data Conversion/Schema migration) – Delta Format Usecases
1. Write customer data into Delta format using overwrite mode
2. Write usage data into Delta format using append mode
3. Write tower data into Delta format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.
6. Compare the parquet location and delta location and try to understand what is the differentiating factor, as both are parquet files only.

##11. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data using saveAsTable() as a managed table
2. Write usage data using saveAsTable() with overwrite mode
3. Drop the managed table and verify data removal
4. Go and check the table overview and realize it is in delta format in the Catalog.
5. Use spark.read.sql to write some simple queries on the above tables created.


##12. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data using insertInto() in a new table and find the behavior
2. Write usage data using insertTable() with overwrite mode

##13. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data into XML format using rowTag as cust
2. Write usage data into XML format using overwrite mode with the rowTag as usage
3. Download the xml data and open the file in notepad++ and see how the xml file looks like.

##14. Compare all the downloaded files (csv, json, orc, parquet, delta and xml) 
1. Capture the size occupied between all of these file formats and list the formats below based on the order of size from small to big.

##15. Do a final exercise of defining one/two liner of... 
1. When to use/benifits csv
2. When to use/benifits json
3. When to use/benifit orc
4. When to use/benifit parquet
5. When to use/benifit delta
6. When to use/benifit xml
7. When to use/benifit delta tables
